In [ ]:
!pip install -U kaleido
import kaleido
import os
import numpy as np
from tqdm import tqdm
from prettytable import PrettyTable
import plotly.graph_objects as go
from scipy.interpolate import interp1d
from collections import Counter

# Loading input data

In [ ]:
shortest_path_distances = np.load('/content/shortest_paths_distances.npy')
shortest_path_nodes = np.load('/content/shortest_paths_nodes.npy')
dimensions = np.load('/content/dimensions.npy')

In [ ]:
longest_path_distances = np.load('/content/longest_paths_distances.npy')
longest_path_nodes = np.load('/content/longest_paths_nodes.npy')
dimensions = np.load('/content/dimensions.npy')

In [ ]:
betweenness_scores = np.load('/content/estimated_betweenness.npy')
normalized_gradient = np.load('/content/normalized_gradient.npy')
normalized_degree = np.load('/content/normalized_degree.npy')
dimensions = np.load('/content/dimensions.npy')

# Shortest Paths Analysis

In [ ]:
shortest_nodes_score = np.zeros(dimensions[0]+dimensions[1]+dimensions[3]+dimensions[5])

for path_index, path in enumerate(shortest_path_nodes):
    # print(f"Path {path_index}:")
    for node_index, node in enumerate(path):
        shortest_nodes_score[node]+=shortest_path_distances[path_index]
        #print(f"  Node {node_index}: {node}")

interp_f = interp1d(list(range(len(shortest_nodes_score))), shortest_nodes_score, kind='linear')
nodes_score_interpolation = interp_f(list(range(len(shortest_nodes_score))))

bar_positions = [dimensions[0], dimensions[0]+dimensions[1], dimensions[0]+dimensions[1]+dimensions[3]]  # Adjust these values as needed

print("\nNodes Score",shortest_nodes_score)
print("Nodes Score Linear Interpolation",nodes_score_interpolation)
print("Separation bars for layers",bar_positions)

In [ ]:
table = PrettyTable()
headers = ['Relative Index','Neuron', 'Score']

shortest_top = np.argpartition(shortest_nodes_score, -40)[-40:]
score = []
for index in shortest_top:
    score.append(shortest_nodes_score[index])

relative_index = []
for index in shortest_top:
    if index < dimensions[0]:
        relative_index.append("0."+str(index))
    elif index < dimensions[0]+dimensions[2]:
        relative_index.append("1."+str(index - dimensions[0]))
    elif index < dimensions[0]+dimensions[2]+dimensions[4]:
        relative_index.append("2."+str(index - dimensions[0]-dimensions[2]))
    else:
        relative_index.append("3."+str(index - dimensions[0]-dimensions[2]-dimensions[4]))

table_data = np.stack((relative_index,shortest_top, score), axis=1)

table.title = "Top 40 Neurons"
table.field_names = headers
table.add_rows(table_data)
table.sortby = "Score"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/top_shortest_neurons.txt', "w") as f:
    f.write(table.get_string())

print(table)

In [ ]:
# Create the histogram trace
fig = go.Figure(data=[go.Bar(x=list(range(len(shortest_nodes_score))), y=shortest_nodes_score, width=1.5, name='Raw data')])

# Add interpolation line
fig.add_trace(go.Scatter(
    x=list(range(len(shortest_nodes_score))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='red',  # Customize line color
    name='Interpolation'
))

# Add vertical bars to the plot
for pos in bar_positions:
    fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Customize the plot (optional)
fig.update_layout(
    title="Histogram of Shortest Paths",
    xaxis_title="Neuron",
    yaxis_title="Distance sum of traversing paths"
)


if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
fig.write_html("./output/shortest_paths.html")

# Save the figure to an image file (e.g., PNG)
fig.write_image("./output/shortest_paths.png")

# Show the plot
fig.show()

In [ ]:
stacked_fig = go.Figure()
stacked_fig.add_trace(go.Scatter(
    x=list(range(len(shortest_nodes_score))),
    y=(nodes_score_interpolation / np.linalg.norm(nodes_score_interpolation)),
    mode='lines',
    line_color='red',  # Customize line color
    name='Normalized Shortest Paths',
    opacity = 0.7
))

# Longest Path Analysis

In [ ]:
longest_nodes_score = np.zeros(dimensions[0]+dimensions[1]+dimensions[3]+dimensions[5])

for path_index, path in enumerate(longest_path_nodes):
    # print(f"Path {path_index}:")
    for node_index, node in enumerate(path):
        longest_nodes_score[node]+=longest_path_distances[path_index]
        #print(f"  Node {node_index}: {node}")

interp_f = interp1d(list(range(len(longest_nodes_score))), longest_nodes_score, kind='linear')
nodes_score_interpolation = interp_f(list(range(len(longest_nodes_score))))

bar_positions = [dimensions[0], dimensions[0]+dimensions[1], dimensions[0]+dimensions[1]+dimensions[3]]  # Adjust these values as needed

print("\nNodes Score",longest_nodes_score)
print("Nodes Score Linear Interpolation",nodes_score_interpolation)
print("Separation bars for layers",bar_positions)

In [ ]:
table = PrettyTable()
headers = ['Relative Index','Neuron', 'Score']

longest_top = np.argpartition(longest_nodes_score, -40)[-40:]
score = []
for index in longest_top:
    score.append(longest_nodes_score[index])

relative_index = []
for index in longest_top:
    if index < dimensions[0]:
        relative_index.append("0."+str(index))
    elif index < dimensions[0]+dimensions[2]:
        relative_index.append("1."+str(index - dimensions[0]))
    elif index < dimensions[0]+dimensions[2]+dimensions[4]:
        relative_index.append("2."+str(index - dimensions[0]-dimensions[2]))
    else:
        relative_index.append("3."+str(index - dimensions[0]-dimensions[2]-dimensions[4]))

table_data = np.stack((relative_index,longest_top, score), axis=1)

table.title = "Top 40 Neurons"
table.field_names = headers
table.add_rows(table_data)
table.sortby = "Score"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/top_longest_neurons.txt', "w") as f:
    f.write(table.get_string())

print(table)

In [ ]:
# Create the histogram trace
fig = go.Figure(data=[go.Bar(x=list(range(len(longest_nodes_score))), y=longest_nodes_score, width=1.5, name='Raw data')])

# Add interpolation line
fig.add_trace(go.Scatter(
    x=list(range(len(longest_nodes_score))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='blue',  # Customize line color
    name='Interpolation'
))

# Add vertical bars to the plot
for pos in bar_positions:
    fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Customize the plot (optional)
fig.update_layout(
    title="Histogram of Longest Paths",
    xaxis_title="Neuron",
    yaxis_title="Distance sum of traversing paths"
)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
fig.write_html("./output/longest_paths.html")

# Save the figure to an image file (e.g., PNG)
fig.write_image("./output/longest_paths.png")

# Show the plot
fig.show()

In [ ]:
stacked_fig.add_trace(go.Scatter(
    x=list(range(len(longest_nodes_score))),
    y=(nodes_score_interpolation / np.linalg.norm(nodes_score_interpolation)),
    mode='lines',
    line_color='blue',  # Customize line color
    name='Normalized Longest Paths',
    opacity = 0.7
))

# Add vertical bars to the plot
for pos in bar_positions:
    stacked_fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Update layout
stacked_fig.update_layout(title='Overlayed Paths')

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
stacked_fig.write_html("./output/overlay_paths.html")

# Save the figure to an image file (e.g., PNG)
stacked_fig.write_image("./output/overlay_paths.png")

# Show the plot
stacked_fig.show()

# Betwenness Analysis

In [ ]:
interp_f = interp1d(list(range(len(betweenness_scores))), betweenness_scores, kind='linear')
nodes_score_interpolation = interp_f(list(range(len(betweenness_scores))))

bar_positions = [dimensions[0], dimensions[0]+dimensions[1], dimensions[0]+dimensions[1]+dimensions[3]]  # Adjust these values as needed

print("Nodes Score",betweenness_scores)
print("Nodes Score Linear Interpolation",nodes_score_interpolation)
print("Separation bars for layers",bar_positions)

In [ ]:
table = PrettyTable()
headers = ['Relative Index','Neuron', 'Betweenness']

betweenness_top = np.argpartition(betweenness_scores, -40)[-40:]
score = []
for index in betweenness_top:
    score.append(betweenness_scores[index])

relative_index = []
for index in betweenness_top:
    if index < dimensions[0]:
        relative_index.append("0."+str(index))
    elif index < dimensions[0]+dimensions[2]:
        relative_index.append("1."+str(index - dimensions[0]))
    elif index < dimensions[0]+dimensions[2]+dimensions[4]:
        relative_index.append("2."+str(index - dimensions[0]-dimensions[2]))
    else:
        relative_index.append("3."+str(index - dimensions[0]-dimensions[2]-dimensions[4]))

table_data = np.stack((relative_index,betweenness_top, score), axis=1)

table.title = "Top 40 Neurons"
table.field_names = headers
table.add_rows(table_data)
table.sortby = "Betweenness"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/top_betweenness_neurons.txt', "w") as f:
    f.write(table.get_string())

print(table)

In [ ]:
# Create the histogram trace
fig = go.Figure(data=[go.Bar(x=list(range(len(betweenness_scores))), y=betweenness_scores, width=1.5, name='Raw data')])

# Add interpolation line
fig.add_trace(go.Scatter(
    x=list(range(len(betweenness_scores))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='green',  # Customize line color
    name='Interpolation'
))

# Add vertical bars to the plot
for pos in bar_positions:
    fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Customize the plot (optional)
fig.update_layout(
    title="Histogram of Betweenness Scores",
    xaxis_title="Neuron",
    yaxis_title="Betweenness Score"
)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
fig.write_html("./output/betweenness.html")

# Save the figure to an image file (e.g., PNG)
fig.write_image("./output/betweenness.png")

# Show the plot
fig.show()

In [ ]:
stacked_fig = go.Figure()
# Add interpolation line
stacked_fig.add_trace(go.Scatter(
    x=list(range(len(betweenness_scores))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='green',  # Customize line color
    name='Betweenness',
    opacity = 0.6
))

# Gradient Analysis

In [ ]:
interp_f = interp1d(list(range(len(normalized_gradient))), normalized_gradient, kind='linear')
nodes_score_interpolation = interp_f(list(range(len(normalized_gradient))))

bar_positions = [dimensions[0], dimensions[0]+dimensions[1], dimensions[0]+dimensions[1]+dimensions[3]]  # Adjust these values as needed

print("\nNodes Score",normalized_gradient)
print("Nodes Score Linear Interpolation",nodes_score_interpolation)
print("Separation bars for layers",bar_positions)

In [ ]:
table = PrettyTable()
headers = ['Relative Index','Neuron', 'Gradient']

gradient_top = np.argpartition(normalized_gradient, -40)[-40:]
score = []
for index in gradient_top:
    score.append(normalized_gradient[index])

relative_index = []
for index in longest_top:
    if index < dimensions[0]:
        relative_index.append("0."+str(index))
    elif index < dimensions[0]+dimensions[2]:
        relative_index.append("1."+str(index - dimensions[0]))
    elif index < dimensions[0]+dimensions[2]+dimensions[4]:
        relative_index.append("2."+str(index - dimensions[0]-dimensions[2]))
    else:
        relative_index.append("3."+str(index - dimensions[0]-dimensions[2]-dimensions[4]))

table_data = np.stack((relative_index,gradient_top, score), axis=1)

table.title = "Top 40 Neurons"
table.field_names = headers
table.add_rows(table_data)
table.sortby = "Gradient"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/top_gradient_neurons.txt', "w") as f:
    f.write(table.get_string())

print(table)

In [ ]:
# Create the histogram trace
fig = go.Figure(data=[go.Bar(x=list(range(len(normalized_gradient))), y=normalized_gradient, width=1.5, name='Raw data')])

# Add interpolation line
fig.add_trace(go.Scatter(
    x=list(range(len(normalized_gradient))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='orange',  # Customize line color
    name='Interpolation'
))

# Add vertical bars to the plot
for pos in bar_positions:
    fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Customize the plot (optional)
fig.update_layout(
    title="Histogram of Normalized Gradient",
    xaxis_title="Neuron",
    yaxis_title="Normalized Gradient"
)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
fig.write_html("./output/normalized_gradient.html")

# Save the figure to an image file (e.g., PNG)
fig.write_image("./output/normalized_gradient.png")

# Show the plot
fig.show()

In [ ]:
# Add interpolation line
stacked_fig.add_trace(go.Scatter(
    x=list(range(len(normalized_gradient))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='orange',  # Customize line color
    name='Gradient',
    opacity = 0.6
))

# Degree Analysis


In [ ]:
interp_f = interp1d(list(range(len(normalized_degree))), normalized_degree, kind='linear')
nodes_score_interpolation = interp_f(list(range(len(normalized_degree))))

bar_positions = [dimensions[0], dimensions[0]+dimensions[1], dimensions[0]+dimensions[1]+dimensions[3]]  # Adjust these values as needed

print("\nNodes Score",normalized_degree)
print("Nodes Score Linear Interpolation",nodes_score_interpolation)
print("Separation bars for layers",bar_positions)

In [ ]:
table = PrettyTable()
headers = ['Relative Index','Neuron', 'Degree']

degree_top = np.argpartition(normalized_degree, -40)[-40:]
score = []
for index in degree_top:
    score.append(normalized_degree[index])

relative_index = []
for index in degree_top:
    if index < dimensions[0]:
        relative_index.append("0."+str(index))
    elif index < dimensions[0]+dimensions[2]:
        relative_index.append("1."+str(index - dimensions[0]))
    elif index < dimensions[0]+dimensions[2]+dimensions[4]:
        relative_index.append("2."+str(index - dimensions[0]-dimensions[2]))
    else:
        relative_index.append("3."+str(index - dimensions[0]-dimensions[2]-dimensions[4]))

table_data = np.stack((relative_index,degree_top, score), axis=1)

table.title = "Top 40 Neurons"
table.field_names = headers
table.add_rows(table_data)
table.sortby = "Degree"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/top_degree_neurons.txt', "w") as f:
    f.write(table.get_string())

print(table)

In [ ]:
# Create the histogram trace
fig = go.Figure(data=[go.Bar(x=list(range(len(normalized_degree))), y=normalized_degree, width=1.5, name='Raw data')])

# Add interpolation line
fig.add_trace(go.Scatter(
    x=list(range(len(normalized_degree))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='violet',  # Customize line color
    name='Interpolation'
))

# Add vertical bars to the plot
for pos in bar_positions:
    fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Customize the plot (optional)
fig.update_layout(
    title="Histogram of Normalized Degree",
    xaxis_title="Neuron",
    yaxis_title="Normalized Degree"
)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
fig.write_html("./output/normalized_degree.html")

# Save the figure to an image file (e.g., PNG)
fig.write_image("./output/normalized_degree.png")

# Show the plot
fig.show()

In [ ]:
# Add interpolation line
stacked_fig.add_trace(go.Scatter(
    x=list(range(len(normalized_degree))),
    y=nodes_score_interpolation,
    mode='lines',
    line_color='violet',  # Customize line color
    name='Degree',
    opacity = 0.6
))

# Add vertical bars to the plot
for pos in bar_positions:
    stacked_fig.add_shape(
        type="line",
        x0=pos,
        y0=0,
        x1=pos,
        y1=fig.data[0].y.max(),
        line=dict(color="black", width=1, dash="dash"),
        name = 'Layers'
    )

# Update layout
stacked_fig.update_layout(title='Overlayed Betweenness, Gradient and Degree')

if not os.path.isdir('./output/'):
    os.makedirs('./output')

# Save the figure to an HTML file
stacked_fig.write_html("./output/overlay_betweenness_gradient_degree.html")

# Save the figure to an image file (e.g., PNG)
stacked_fig.write_image("./output/overlay_betweenness_gradient_degree.png")

# Show the plot
stacked_fig.show()

# Find common nodes in previous analysis

In [ ]:
all_top = shortest_top + longest_top + betweenness_top + gradient_top + degree_top
occurrences = Counter(all_top)

table = PrettyTable()
headers = ['Relative Index','Neuron', 'Occurrences']

table.title = "Neuron Occurrences in top 40"
table.field_names = headers

for value, count in occurrences.items():
   if value < dimensions[0]:
        table.add_row(["0."+str(value) ,value, count])
   elif value < dimensions[0]+dimensions[2]:
        table.add_row(["1."+str(value - dimensions[0]), value, count])
   elif value < dimensions[0]+dimensions[2]+dimensions[4]:
        table.add_row(["2."+str(value - dimensions[0]-dimensions[2]), value, count])
   else:
        table.add_row(["3."+str(value - dimensions[0]-dimensions[2]-dimensions[4]), value, count])

table.sortby = "Occurrences"
table.reversesort = True

if not os.path.isdir('./output/'):
    os.makedirs('./output')

with open('./output/neuron occurrences.txt', "w") as f:
    f.write(table.get_string())

print(table)

# Compress Output

In [ ]:
!zip -r /content/Analysis_Result.zip /content/output